---
title: "desafio12"
format: html
editor: visual
---

In [ ]:
import polars as pl
import sqlite3

In [ ]:
conn = sqlite3.connect('//smb/ra185325/Documentos/dados/data.db')
cursor = conn.cursor()

In [ ]:
cursor.execute('''
CREATE TABLE vendas (
    id INTEGER PRIMARY KEY,
    vendedor TEXT,
    produto TEXT,
    valor REAL,
    data_venda DATE
)
''')

In [ ]:
cursor.execute('''
INSERT INTO vendas (vendedor, produto, valor, data_venda)
VALUES
    ('Ana', 'Produto A', 120.5, '2024-09-01'),
    ('Carlos', 'Produto B', 200.0, '2024-10-02'),
    ('Ana', 'Produto C', 150.0, '2024-09-03'),
    ('Bruno', 'Produto A', 300.0, '2024-11-04'),
    ('Carlos', 'Produto C', 100.0, '2024-10-05');
''')
conn.commit()

In [ ]:
cursor.execute("SELECT * FROM vendas")
rows = cursor.fetchall()
for row in rows:
  print(row)

In [ ]:
import polars as pl
dados = pl.read_database("SELECT * FROM vendas", conn)
print(dados)

In [ ]:
vendas_total = pl.read_database('''
   SELECT vendedor, SUM(valor) as total_vendas
   FROM vendas
   GROUP BY vendedor;
''', conn)
print(vendas_total)

In [ ]:
vendas_medias = pl.read_database('''
   SELECT vendedor, AVG(valor) as total_vendas
   FROM vendas
   GROUP BY vendedor;
''', conn)
print(vendas_medias)

In [ ]:
vendas_comb = pl.read_database("""
SELECT vendedor, 
       COUNT(*) as numero_vendas,
       SUM(valor) as total_vendas, 
       AVG(valor) as media_vendas
FROM vendas
GROUP BY vendedor;
""", conn)

In [ ]:
print(vendas_comb)

In [ ]:
ticket_alto = pl.read_database("""
SELECT * FROM vendas WHERE valor >= 200
""", conn)
print(ticket_alto)

In [ ]:
vendas_mensais = pl.read_database("""
SELECT strftime('%Y-%m', data_venda) AS mes, SUM(valor) AS total_vendas
FROM vendas GROUP BY mes ORDER BY mes
""", conn)
print(vendas_mensais)

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS produtos (
    id INTEGER PRIMARY KEY,
    nome TEXT NOT NULL,
    categoria TEXT NOT NULL,
    preco REAL NOT NULL,
    estoque INTEGER NOT NULL
);
''')
cursor.execute('''
INSERT INTO produtos (nome, categoria, preco, estoque) VALUES
    ('Produto A', 'Categoria 1', 100.0, 50),
    ('Produto B', 'Categoria 2', 150.0, 30),
    ('Produto C', 'Categoria 1', 200.0, 20),
    ('Produto D', 'Categoria 2', 250.0, 10),
    ('Produto E', 'Categoria 3', 300.0, 0);
''')
conn.commit()

In [ ]:
prods = pl.read_database("SELECT * FROM produtos", conn)
print(prods)

In [ ]:
lucros = pl.read_database("""
SELECT produto, valor AS compra, preco AS venda, preco-valor AS lucro
FROM vendas
INNER JOIN produtos ON vendas.produto = produtos.nome
""", conn)
print(lucros)

In [ ]:
lucro_medio = pl.read_database("""
SELECT vendedor, produto, AVG(preco-valor) AS lucro_medio
FROM vendas
INNER JOIN produtos ON vendas.produto = produtos.nome
GROUP BY vendedor
""", conn)
print(lucro_medio)

In [ ]:
from datetime import datetime

# Pega a data and hora atuais e formata para um texto legível
timestamp = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

print(f"Arquivo compilado em: {timestamp}")